In [1]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

from sklearn import metrics
from sklearn.metrics import roc_auc_score

# import matplotlib.pyplot as plt

from sklearn.externals import joblib

In [2]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
# from hyperopt.mongoexp import MongoTrials

In [3]:
import matplotlib.pyplot as plt

In [4]:
from sklearn.metrics import accuracy_score

In [5]:
import pandas as pd
import numpy as np

In [6]:
import pickle

In [7]:
import time

### Model trainig with hyper-parameter tuning

In [8]:
## Model developed to be used in for loop without print statement

def model_fit_hyperopt(params):
    
#     attempt_count += 1
    
    global num
    f1 = open("status.txt", "w")
    f1.write("num:{}".format(num))
    f1.close() 
    num = num +1

    ### Asssigning default parameters
#     useTrainCV = False
    cv_folds = 5
    early_stopping_rounds = 50
    
    ### Forming copy of input datasets
    train_c, valid_c = train_sample.copy(), valid_sample.copy()
#     out_path = params['out_path']
    predictors, target = list(params['predictors']), params['target']
    
    alg = XGBClassifier(
                learning_rate =params['learning_rate'],
                n_estimators=5000,
                max_depth=params['max_depth'],
                min_child_weight=params['min_child_weight'],
                gamma=params['gamma'],
                subsample=params['subsample'],
                colsample_bytree=params['colsample_bytree'],
                objective= 'binary:logistic',
                scale_pos_weight=params['scale_pos_weight'],
                seed=27, n_jobs=7)

#     if useTrainCV:
#         xgb_param = alg.get_xgb_params()
#         xgtrain = xgb.DMatrix(train_c[predictors].values, label=train_c[target].values)
#         xgtest = xgb.DMatrix(valid_c[predictors].values)
#         cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
#             metrics = 'auc', early_stopping_rounds=early_stopping_rounds, verbose_eval=False, seed=27)
#         ne_new = n_estimators=cvresult.shape[0]
#         alg.set_params(n_estimators=cvresult.shape[0])
        
    #Fit the algorithm on the data
    eval_set = [(valid_c[predictors],valid_c[target])]
    alg.fit(train_c[predictors], train_c[target],eval_metric='auc', early_stopping_rounds=early_stopping_rounds, eval_set = eval_set, verbose = True)     
    
    valid_c.loc[:,'prob'] = alg.predict_proba(valid_c[predictors])[:,1]
#     test_c.loc[:,'prob'] = alg.predict_proba(test_c[predictors])[:,1]
    
#     valid_auc = roc_auc_score(np.array(valid_c.loc[:,target]), np.array(valid_c.loc[:,'prob']))
#     test_auc = roc_auc_score(np.array(test_c.loc[:,target]), np.array(test_c.loc[:,'prob']))
    loss = 1 - valid_auc
    del train_c
    del valid_c
#     del test_c
#     if out_path != None:
#         f = open(os.path.join(out_path, "hyperopt_logs.txt"), "a+")
#         f.write("Valid AUC: {} and Test AUC: {}, loss: {}".format(valid_auc,test_auc,loss))
#         f.close()
    
#     print ("Valid AUC: {} and Test AUC: {}, Loss: {}".format(valid_auc, test_auc, loss))
    
    return {'loss': loss, 'status': STATUS_OK, 'params': params, 'model':alg}

In [9]:
def get_best_model_xgb(space):
    
#     f = open(os.path.join(out_path, "hyperopt_logs.txt"), "w+")
#     f.close()

    
    trials=Trials()
#     trials = MongoTrials('mongo://localhost:27017/foo_db/jobs', exp_key='exp1')
    global num
    num = 1
    best = fmin(model_fit_hyperopt, space, algo=tpe.suggest, max_evals=50, trials=trials)
    
    return trials.best_trial['result']['model'], trials.best_trial['result']

In [10]:
def data(df,target):
    processed_df = pd.get_dummies(df,sparse=True)
    df_features = list(set(processed_df.columns) - set([target]))
#     train, valid, test = train_validate_test_split(processed_df)
    train = processed_df.ix[train_o.index]
    valid = processed_df.ix[valid_o.index]
    test = processed_df.ix[test_o.index]
    return train, valid, test, df_features, target

In [11]:
train_o = pd.read_csv('../../data/train_insurance.csv',index_col = 0)
valid_o = pd.read_csv('../../data/valid_insurance.csv',index_col = 0)
test_o = pd.read_csv('../../data/test_insurance.csv',index_col = 0)

In [12]:
all_data = pd.concat([train_o,valid_o,test_o])

In [13]:
all_data.drop(['NewEstimateTotal','EstimateTotal'],axis=1,inplace=True)

In [14]:
train, valid, test, df_features, df_target = data(all_data,target='StartedFlag')
del all_data
train_sample = train.head(100000)
valid_sample = valid.head(20000)
del train
del valid
del train_o
del valid_o
del test_o
del test

/home/ubuntu/insurance_izenda_analysis/insurace_izenda/lib/python3.5/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
/home/ubuntu/insurance_izenda_analysis/insurace_izenda/lib/python3.5/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/ubuntu/insurance_izenda_analysis/insurace_izenda/lib/python3.5/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-

In [15]:
start_time = time.time()
space = {'learning_rate': hp.choice('learning_rate',[0.01, 0.02, 0.04, 0.06, 0.08, 0.1, 0.2, 0.3, 0.4, 0.6, 0.8, 1]),
                     'min_child_weight': hp.choice('min_child_weight', [0,1,2,3,4,5,6,7]),
                     'max_depth': hp.choice('max_depth', [3,4,5,6,7,8,9,10]),
                     'gamma': hp.choice('gamma', [0,1,5,10]),
                     'subsample': hp.choice('subsample',np.arange(0.4,1.05,0.1)),
                     'colsample_bytree': hp.choice('colsample_bytree',np.arange(0.5,1.05,0.1)),
                     'scale_pos_weight': (len(train_sample) -train_sample[df_target].sum())/train_sample[df_target].sum(),
                     'predictors': df_features,
                     'target': df_target
        }

best_model, best_trails1 = get_best_model_xgb(space)

# To save model
with open('insurance_classify_100000_started_flag_hyperopt.pkl', 'wb') as f:
    pickle.dump(best_model, f)


f = open("started_flag_100000_hyperopt.txt", "w")
f.write("Took {} seconds with best trails as: {}".format(time.time()-start_time,best_trails1))
f.close() 

[0]	validation_0-auc:0.817543
Will train until validation_0-auc hasn't improved in 50 rounds.
[1]	validation_0-auc:0.821135
[2]	validation_0-auc:0.824302
[3]	validation_0-auc:0.825705
[4]	validation_0-auc:0.826489
[5]	validation_0-auc:0.826886
[6]	validation_0-auc:0.827771
[7]	validation_0-auc:0.827585
[8]	validation_0-auc:0.826947
[9]	validation_0-auc:0.828072
[10]	validation_0-auc:0.829313
[11]	validation_0-auc:0.829832
[12]	validation_0-auc:0.830169
[13]	validation_0-auc:0.830573
[14]	validation_0-auc:0.832405
[15]	validation_0-auc:0.833583
[16]	validation_0-auc:0.834674
[17]	validation_0-auc:0.835298
[18]	validation_0-auc:0.836112
[19]	validation_0-auc:0.83673
[20]	validation_0-auc:0.83766
[21]	validation_0-auc:0.838519
[22]	validation_0-auc:0.839017
[23]	validation_0-auc:0.839202
[24]	validation_0-auc:0.839589
[25]	validation_0-auc:0.840143
[26]	validation_0-auc:0.840711
[27]	validation_0-auc:0.840823
[28]	validation_0-auc:0.841503
[29]	validation_0-auc:0.84163
[30]	validation_0-a

[259]	validation_0-auc:0.870913
[260]	validation_0-auc:0.87093
[261]	validation_0-auc:0.870976
[262]	validation_0-auc:0.871007
[263]	validation_0-auc:0.871032
[264]	validation_0-auc:0.871067
[265]	validation_0-auc:0.871183
[266]	validation_0-auc:0.871352
[267]	validation_0-auc:0.871398
[268]	validation_0-auc:0.87144
[269]	validation_0-auc:0.87149
[270]	validation_0-auc:0.871509
[271]	validation_0-auc:0.871533
[272]	validation_0-auc:0.871527
[273]	validation_0-auc:0.871559
[274]	validation_0-auc:0.871593
[275]	validation_0-auc:0.871623
[276]	validation_0-auc:0.871656
[277]	validation_0-auc:0.871703
[278]	validation_0-auc:0.871729
[279]	validation_0-auc:0.871728
[280]	validation_0-auc:0.87174
[281]	validation_0-auc:0.871738
[282]	validation_0-auc:0.871759
[283]	validation_0-auc:0.871818
[284]	validation_0-auc:0.871849
[285]	validation_0-auc:0.871895
[286]	validation_0-auc:0.871884
[287]	validation_0-auc:0.871906
[288]	validation_0-auc:0.871922
[289]	validation_0-auc:0.871963
[290]	valida

[516]	validation_0-auc:0.875107
[517]	validation_0-auc:0.875107
[518]	validation_0-auc:0.875107
[519]	validation_0-auc:0.875109
[520]	validation_0-auc:0.875109
[521]	validation_0-auc:0.875109
[522]	validation_0-auc:0.875109
[523]	validation_0-auc:0.875109
[524]	validation_0-auc:0.875174
[525]	validation_0-auc:0.875174
[526]	validation_0-auc:0.875174
[527]	validation_0-auc:0.875202
[528]	validation_0-auc:0.875202
[529]	validation_0-auc:0.875202
[530]	validation_0-auc:0.875202
[531]	validation_0-auc:0.875197
[532]	validation_0-auc:0.875197
[533]	validation_0-auc:0.875197
[534]	validation_0-auc:0.875197
[535]	validation_0-auc:0.875197
[536]	validation_0-auc:0.875197
[537]	validation_0-auc:0.875197
[538]	validation_0-auc:0.875197
[539]	validation_0-auc:0.875197
[540]	validation_0-auc:0.875235
[541]	validation_0-auc:0.875236
[542]	validation_0-auc:0.875305
[543]	validation_0-auc:0.875322
[544]	validation_0-auc:0.875367
[545]	validation_0-auc:0.875367
[546]	validation_0-auc:0.875429
[547]	va

[773]	validation_0-auc:0.876408
[774]	validation_0-auc:0.876408
[775]	validation_0-auc:0.876408
[776]	validation_0-auc:0.876408
[777]	validation_0-auc:0.876408
[778]	validation_0-auc:0.876408
[779]	validation_0-auc:0.876408
[780]	validation_0-auc:0.876408
[781]	validation_0-auc:0.876414
[782]	validation_0-auc:0.876423
[783]	validation_0-auc:0.876432
[784]	validation_0-auc:0.876432
[785]	validation_0-auc:0.876432
[786]	validation_0-auc:0.876432
[787]	validation_0-auc:0.876432
[788]	validation_0-auc:0.876438
[789]	validation_0-auc:0.876464
[790]	validation_0-auc:0.876464
[791]	validation_0-auc:0.876464
[792]	validation_0-auc:0.876464
[793]	validation_0-auc:0.876481
[794]	validation_0-auc:0.876481
[795]	validation_0-auc:0.876481
[796]	validation_0-auc:0.876481
[797]	validation_0-auc:0.876481
[798]	validation_0-auc:0.876481
[799]	validation_0-auc:0.876481
[800]	validation_0-auc:0.876481
[801]	validation_0-auc:0.876481
[802]	validation_0-auc:0.876481
[803]	validation_0-auc:0.876481
[804]	va

[1029]	validation_0-auc:0.87691
[1030]	validation_0-auc:0.87691
[1031]	validation_0-auc:0.876925
[1032]	validation_0-auc:0.876925
[1033]	validation_0-auc:0.876938
[1034]	validation_0-auc:0.876938
[1035]	validation_0-auc:0.876938
[1036]	validation_0-auc:0.876938
[1037]	validation_0-auc:0.876944
[1038]	validation_0-auc:0.876944
[1039]	validation_0-auc:0.876978
[1040]	validation_0-auc:0.876978
[1041]	validation_0-auc:0.876978
[1042]	validation_0-auc:0.876978
[1043]	validation_0-auc:0.876978
[1044]	validation_0-auc:0.876978
[1045]	validation_0-auc:0.876978
[1046]	validation_0-auc:0.876978
[1047]	validation_0-auc:0.876978
[1048]	validation_0-auc:0.876988
[1049]	validation_0-auc:0.876988
[1050]	validation_0-auc:0.876988
[1051]	validation_0-auc:0.876988
[1052]	validation_0-auc:0.876988
[1053]	validation_0-auc:0.876988
[1054]	validation_0-auc:0.876988
[1055]	validation_0-auc:0.877008
[1056]	validation_0-auc:0.877008
[1057]	validation_0-auc:0.877008
[1058]	validation_0-auc:0.877008
[1059]	valid

NameError: name 'loss' is not defined

In [ ]:
# To load the pickle model\n",
with open('insurance_classify_100000_estimate_total.pkl', 'rb') as f:
    model = pickle.load(f)

In [20]:
valid.loc[:,'prob_up'] = model.predict_proba(valid[df_features])[:,1]

valid['prediction'] = np.where(valid.prob_up > 0.5,1,0)

accuracy_score(valid.StartedFlag, valid.prediction)

0.82179240007352

In [21]:
test.loc[:,'prob_up'] = model.predict_proba(test[df_features])[:,1]

test['prediction'] = np.where(test.prob_up > 0.5,1,0)

accuracy_score(test.StartedFlag, test.prediction)

0.8212032927282326

In [25]:
# Confusion Matrix on Test Data
metrics.confusion_matrix(test.StartedFlag, test.prediction)

array([[ 95387,  19929],
       [ 25792, 114607]])